<a href="https://colab.research.google.com/github/sunny0103/DeepLearning_nlp_projects/blob/main/News_group/News_group_classification_hyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
cd "/content/drive/MyDrive/Data/news_group"

/content/drive/MyDrive/Data/news_group


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

## Clean train and test data

In [4]:
# train = pd.read_csv('./train.csv')
# test = pd.read_csv('./test.csv')

In [5]:
# train.head()

In [6]:
# def checking_missing_col(dataframe):
#   missing_col = []
#   for col in dataframe.columns:
#     missing_values = sum(dataframe[col].isna())
#     is_missing = True if missing_values>=1 else False
#     if is_missing:
#       print(f"결측치가 있는ㄴ 컬럼은 {col}입니다.")
#       print(f"해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.")
#   if missing_col == []:
#     print("결측치가 존재하지 않습니다.")
#   return missing_col

# missing_col = checking_missing_col(train)

In [7]:
# unique_target = train['target'].unique()
# sorted(unique_target)

In [8]:
# group_div = train.groupby(['target']).agg('count')
# group_div

In [9]:
# import re

# def clean_text(texts):
#   corpus = []
#   for i in range(0,len(texts)):
#     review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"\n\]\[\>\<]', '',texts[i]) #@%*=()/+ 와 같은 문장부호 제거
#     review = re.sub(r'\d+','', review)#숫자 제거
#     review = review.lower() #소문자 변환
#     review = re.sub(r'\s+', ' ', review) #extra space 제거
#     review = re.sub(r'<[^>]+>','',review) #Html tags 제거
#     review = re.sub(r'\s+', ' ', review) #spaces 제거
#     review = re.sub(r"^\s+", '', review) #space from start 제거
#     review = re.sub(r'\s+$', '', review) #space from the end 제거
#     review = re.sub(r'_', ' ', review) #space from the end 제거
#     corpus.append(review)
#   return corpus

In [10]:
# from copy import deepcopy
# dc_train = deepcopy(train)
# temp =clean_text(dc_train['text'])
# dc_train['text'] = temp
# dc_train.to_csv('./cleaned_train.csv', index=False)

In [11]:
# tm = pd.read_csv('./cleaned_train.csv')
# tm.text

In [12]:
# dc_test = deepcopy(test)
# temp =clean_text(dc_test['text'])
# dc_test['text'] = temp
# dc_test.to_csv('./cleaned_test.csv')

use the cleaned train and test data

In [13]:
!pip install transformers[torch] datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00


In [14]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.1 MB/s eta 0:00:00


In [15]:
from sklearn.model_selection import train_test_split

from transformers import (AutoTokenizer,
                          AdamW,
                          AutoModelForSequenceClassification)

from datasets import load_dataset

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn

import evaluate

from tqdm.notebook import tqdm

import optuna

In [16]:
def seed_everything(seed: int = 42):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [17]:
train = pd.read_csv('./cleaned_train.csv')
train.head()

,id,text,target
0,0,they were and even if washington might conside...,10
1,1,we run spacenews views on our stareach bbs a l...,14
2,2,not to worry the masons have been demonized an...,19
3,3,only brendan mckay or maybe arf would come to ...,17
4,4,help i am running some sample problems from or...,5


In [18]:
target_list = sorted(list(set(train['target'])))
target_list

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [19]:
# MODEL_NAME = "distilbert-base-uncased"
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [20]:
train_set, valid_set =train_test_split(train, test_size=0.2)

In [21]:
# #sample train
train_set = train_set.sample(n=2000)
valid_set = valid_set.sample(n=500)

In [22]:
max_len = 512
# Convert the NumPy array to a list
train_texts = train_set['text'].astype(str).values.tolist()
valid_text = valid_set['text'].astype(str).values.tolist()

encoded_data_train = tokenizer.batch_encode_plus(
    train_texts,  # Use the list here
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=max_len,
    return_tensors='pt'
)

encoded_data_valid = tokenizer.batch_encode_plus(
    valid_text,
    add_special_tokens = True,
    return_attention_mask = True,
    pad_to_max_length = True,
    max_length = max_len,
    return_tensors ='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_set.target.values)

input_ids_val = encoded_data_valid['input_ids']
attention_masks_val = encoded_data_valid['attention_mask']
labels_val = torch.tensor(valid_set.target.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [24]:
train_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train)
valid_dataset = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [25]:
BATCH_SIZE = 16

trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [26]:
def compute_metrics(pred, label):
  return sum(pred==label)/len(label)

In [27]:
id2label = {0 : 'alt.atheism', 1: 'comp.graphics',2 : 'comp.os.ms-windows.misc',
           3 : 'comp.sys.ibm.pc.hardware', 4 : 'comp.sys.mac.hardware',
           5 : 'comp.windows.x', 6 : 'misc.forsale', 7: 'rec.autos',
           8 : 'rec.motorcycles', 9 : 'rec.sport.baseball', 10 : 'rec.sport.hockey',
           11 : 'sci.crypt', 12 : 'sci.electronics', 13 : 'sci.med',
           14 : 'sci.space', 15 : 'soc.religion.christian', 16 : 'talk.politics.guns',
           17 : 'talk.politics.mideast', 18 : 'talk.politics.misc', 19 : 'talk.religion.misc'}


label2id = {'alt.atheism': 0, 'comp.graphics': 1, 'comp.os.ms-windows.misc': 2,
            'comp.sys.ibm.pc.hardware': 3, 'comp.sys.mac.hardware': 4, 'comp.windows.x': 5,
            'misc.forsale': 6, 'rec.autos': 7, 'rec.motorcycles': 8, 'rec.sport.baseball': 9,
            'rec.sport.hockey': 10, 'sci.crypt': 11, 'sci.electronics': 12, 'sci.med': 13,
            'sci.space': 14, 'soc.religion.christian': 15,'talk.politics.guns': 16,
            'talk.politics.mideast': 17, 'talk.politics.misc': 18, 'talk.religion.misc': 19}


In [28]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,
                                                           num_labels=len(target_list),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
import torch, gc

In [30]:
class Trainer:
  def __init__(self, trainloader, validloader, model, optimizer, criterion, device):
    self.trainloader = trainloader
    self.validloader = validloader
    self.model = model
    self.optimizer = optimizer
    self.criterion = criterion
    self.device = device

  def train(self, epochs=1):
    self.model.train()
    for epoch in range(1, epochs+1):
      running_loss = 0.0
      for input_ids, attention_mask, y in tqdm(self.trainloader):
        input_ids = input_ids.to(self.device)
        attention_mask = attention_mask.to(self.device)
        y = y.to(self.device)

        self.optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)[0]
        loss = self.criterion(outputs, y)
        loss.backward()
        self.optimizer.step()
        running_loss += loss.item()
      train_loss = running_loss/len(self.trainloader)
      valid_loss , predictions, labels = self.validate()
      accuracy = compute_metrics(predictions, labels)
      tqdm.write('Epoch:{:1d}, Train Loss: {:.3f}, Validation Loss: {:.3f}, Accuracy: {:.3f}'.format(
          epoch, train_loss, valid_loss, accuracy))
      running_loss = 0.0
    return accuracy

  def validate(self):
    self.model.eval()
    predictions, labels = [], []
    with torch.no_grad():
      running_loss = 0.0
      for input_ids, attention_mask, y in tqdm(self.validloader):
        input_ids = input_ids.to(self.device)
        attention_mask = attention_mask.to(self.device)
        y = y.to(self.device)

        outputs = model(input_ids, attention_mask)[0]
        loss = self.criterion(outputs, y).to(self.device)
        running_loss += loss.item()

        pred = outputs.detach().cpu().numpy().argmax(1)
        label = y.detach().cpu().numpy()
        predictions.append(pred)
        labels.append(label)

      valid_loss = running_loss/len(self.validloader)
      predictions =  np.concatenate(predictions, axis=0)
      labels = np.concatenate(labels, axis=0)
      return valid_loss, predictions, labels

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [32]:
def objective(trial):
  params ={'Epochs': trial.suggest_int('Epochs', 3, 7, 1),
           'lr': trial.suggest_loguniform('lr', 2e-7, 2e-1)
           }
  criterion =  nn.CrossEntropyLoss().to(device)
  optimizer = AdamW(model.parameters(), lr = params['lr'])
  trainer = Trainer(trainloader= trainloader,
                validloader = validloader,
                model = model,
                optimizer = optimizer,
                criterion = criterion,
                device = device)
  valid_acc = trainer.train(epochs = params['Epochs'])
  return valid_acc

In [33]:
sampler = optuna.samplers.TPESampler()
study = optuna.create_study(study_name ='distill_parameter_opt',
                            direction ='maximize',
                            sampler = sampler)
study.optimize(objective, n_trials=20)
print("Best accuracy", study.best_value)
print("Best trial", study.best_trial.params)

[I 2023-09-20 05:59:44,676] A new study created in memory with name: distill_parameter_opt
<ipython-input-32-fc3574224ea0>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lr': trial.suggest_loguniform('lr', 2e-7, 2e-1)
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch:1, Train Loss: 2.715, Validation Loss: 2.217, Accuracy: 0.440


  0%|          | 0/125 [00:00<?, ?it/s]

...

Epoch:5, Train Loss: 2.986, Validation Loss: 2.994, Accuracy: 0.048


  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

[I 2023-09-20 06:49:01,654] Trial 19 finished with value: 0.048 and parameters: {'Epochs': 6, 'lr': 1.438975607835455e-05}. Best is trial 0 with value: 0.658.


Epoch:6, Train Loss: 2.986, Validation Loss: 2.992, Accuracy: 0.048
Best accuracy 0.658
Best trial {'Epochs': 4, 'lr': 1.409014093743804e-05}


In [34]:
optuna.visualization.plot_parallel_coordinate(study)

In [35]:
optuna.visualization.plot_param_importances(study)